## ATTPP (Actor-based Trait/Token Programming Paradigm)
I've come up with this pattern for PL. The idea is to have a set() of flags - called traits that define the properties and methods of a class. Instead of defining "classes" of traits or subclassing explicitely, the traits are grouped into so-called "stereotypes", simple set()s of traits that can interact with each other.

Traits basically implement the flyweight and visitor pattern while simplifying the handling on the user side, overcoming all the problems of multiple inheritance.

There are some issues here:

### Traits as enums?
The first approach used a Trait(Enum) with name->Implementation, which seemed to work at first but being an enum, this mapping could never be changed, but worse: it never can be extended while all traits using the same implementation map to the same name.

### Traits as classes?
Classes in module-namespace act like enums (they are singletons), but unlike enums, their implementation can be changed on the fly and classes can be added.
This may be a feasible solution, but it adds the temptation to have state in the Trait() which actually belongs to the Thing() and should be shared with other traits, adding unnecessary complexity.

### Traits as static methods?
If we accept that traits should have no state by itself, we could take the functional route and implement them as externalized methods of Thing. This can be done by having a Trait class which just acts as a namespace for @staticmethods, taking Thing() as parameter that can be introspected. The Thing.bag is a very powerful datastructure that can be used to establish locks and loops across traits.

### Actors!
Almost forgot the purpose of the exercise: to separate the implementation from the Thing to easily switch it on the fly, depending on the context. So, all we need to do is to go with the first approach - traits as enums - but then have an Actor/Visitor class that implements the details, simply checking the thing for traits and tokens. It's really simple and elegant even. As traits and tokens are enums, their namespaces are seperated, typos are caught instantly and the resulting code is clearly readable with a minimum of noise. 

If there are things that should be stored with a thing but can't easily be coded as a trait or token (for instance the coordinates of a portal), it's best to have an external WeakKeyDictionary store the data associated with the thing (flyweight pattern) so that traits and tokens can be used as "metadata".

## FSM!
Implementing finite state machines with enums really is cool and simple, as shown elsewhere. To implement complex kinds of FSM you need some kinds of tokens - but wait! Traits are enums and they work in tandem with tokens! This smells like a delicious cake of pattern-goodness.

In [ ]:
from enum import Enum
from collections import Counter

class Trait(Enum):
    foo = 1
    bar = 2
    baz = 3

class Token(Enum):
    amount = 1

class Stereotype(set):
    pass

a = Stereotype([Trait.foo, Trait.bar])
b = Stereotype([Trait.bar])
c = Stereotype([Trait.bar, Trait.baz])

class Actor:
    def do_foo(self, thing):
        print("foo!")
        
    def examine(self, thing):
        if Trait.foo in thing.traits:
            self.do_foo(thing)

class Thing:
    __slots__ = ["__weakref__", "traits", "bag"]
    
    def __init__(self, 
                 stereotypes=[], in_traits=set(), ex_traits=set()):
        """First set up a pile of traits (with stereotypes), then add
        all the traits that should be included. At the end,
        polish the rough edges by taking away the traits that should be 
        excluded.
        """
        self.bag = Counter()
        self.traits = {t for S in stereotypes for t in S}
        self.traits.update(set(in_traits))
        self.traits.difference_update(set(ex_traits))
        
    def accept(self, actor):
        actor.examine(self)
        
obj = Thing([a,c])
actor = Actor()

class Container(Thing):
    def __init__(self, isolated_traits=[], contents=[], *args):
        super().__init__(*args)
        self.isolated_traits = isolated_traits
        self.contents = contents
        self.entailed_traits = {}
        for thing in contents:
            for trait in thing.traits:
                self.entailed_traits.setdefault(trait, []).append(thing)

chest = Container(contents=[obj])
chest.entailed_traits

## pluggable adapter, delegator or trait with flyweight
this is an exceptionally useful combination of patterns for several reasons:
firstly, no subclassing is necessary. this means there are no collisions of methods and attributes, reducing complexity a lot. It also means Entities and delegators can easily be subclassed for other reaons as long as the method signatures are compatible (kwargs, not positional for cooperative super()!). 
use NotImplemented instead of NotImplementedError for quick check without exceptions. the delegation class should be just a set of related methods with no internal state. this should enable easy functional testing of the delegation with no side effects. last but not least, handling the delegation as a property allows quick and uncomplicated hotplugging of delegators with no change of call-semantics whatsoever. this means the calling side can't and shouldn't care about the current implementation (it could change at any time!), enforcing loose coupling in calling. this also helps in keeping the code at the calling side real simple since only the Entity with a property is called. last but not least, extra needed state associated with the delegator can easily be stored outside in a WeakKeyDict or WeakValueDict so that no unnecessary housekeeping, overcrowding of the Entity namespace or internal state inside the delegator is needed, enabling the flyweight pattern (very little internal state, no \__dict\__, and \__slots\__ instead).

In [ ]:
from weakref import WeakKeyDictionary

positions = WeakKeyDictionary()
movers = WeakKeyDictionary()


class Entity:
    slots = ["weakref", "mover", "position"]
    def _mover():
        def fget(self):
            return movers.get(self, None)
        def fset(self, value):
            movers[self] = value
            positions[self] = 0
        def fdel(self):
            del movers[self]
            del positions[self]
        return locals()
    mover = property(**_mover())
    del _mover
    
    @property
    def position(self):
        return positions.get(self, 0)

    def move(self):
        if self.mover is not None:
            self.mover.move(self)
        else:
            return NotImplemented

class Mover:
    def move(self):
        positions[self] += 1

e = Entity()
print("position", e.position)
print(e.move())
e.mover = Mover
print(e.move())
print("position", e.position)

In [ ]:
from collections import deque, defaultdict
from functools import partial

class CallableDeque(deque):
    def __call__(self):
        notes = defaultdict(lambda: 0)
        for f in self:
            notes = f(notes)
        return dict(notes)
    def append(self, func, source, target):
        super().append(partial(func, source, target))
        
    
d = CallableDeque()


class Buff:
    @staticmethod
    def strength(source, target, notes):
        notes["attack_modifier"] += 1
        return notes

class Debuff:
    @staticmethod
    def weakness(source, target,notes):
        return notes

class Entity:
    pass

attacker = Entity()
defender = Entity()

d.append(Buff.strength, attacker, attacker)
d.append(Debuff.weakness, defender, attacker)

d()

## SSCCE for trait-token paradigm programming 
(Short, Self Contained, Correct (Compilable), Example)
This is the simplest example I could come up with to illustrate the trait-oriented programming pattern I developed.

1. First, an enum is declared as a contract between all parties involved. All parties promise to use *only* the enum as traits **nothing else**. 
2. The Thing class only has traits (as a set of enum) to define behaviour and tokens (as a counter of enum) for a minimum of state. If more state is needed, it can/should be externalized via properties operating on global dicts.
3. Any behaviour is externalized into Actor classes or simple handler functions with no state. They are envoked from an accept function in the Thing class, which passes the instance to the Actor/handler. The Actor/handler introspects the traits and tokens of the thing and operates accordingly.

This clearly separates state from functionality: Thing represents state (traits, counter and interface to global dicts), Actor implements functionality without state.
The first and obvious advantage of this pattern is that it is trivial to expand and recombine functionality on the fly without hardcoding classes and subclasses. Since traits are realized as sets of enums, handling of "classes" is simple, safe and fast. This extraordinary combination of qualities is hard to achieve otherwise.

Another obvious advantage of this pattern is that it implements the foundation of finite state machines (FSM), making it trivial to realize any type of any complexity of state machine from here.

## Linchpin: case handling
The pattern stands or falls depending on how cases are handled in the Actor class, realizing the traits. If there are many cases (more than 2), it is advisable to use the dictionary-switch-pattern. Depending on how the traits actually should be handled, it's possible to use tuples of bool as keys to identify cases and test against traits once (that's one of the most efficient ways). This is recommended for traits that interact. If there are traits that are independent of each other, they can be handled in a for-loop over the dict of cases, this time with frozensets of traits to test against.
Since traits are implemented as sets, many complex tests (for instance supersets) can be realized efficiently without making the code complicated.

In [ ]:
from enum import Enum
from collections import Counter

class Trait(Enum):
    foo = 1
    bar = 2
    
class Thing:
    __slots__ = ["traits", "tokens", "__weakref__"]
    def __init__(self, traits):
        self.traits = set(traits)
        self.tokens = Counter()
    
    def accept(self, handle):
        handle(self)
    
class Actor:
    def __call__(self, thing):
        cases =  {(False, False): lambda: print("case 1"),
                  (False, True): lambda: print("case 2"),
                  (True, False): lambda: print("case 3"),
                  (True, True): lambda: print("case 4"),
                 }
        cases[(Trait.foo in thing.traits, Trait.bar in thing.traits)]()

t = Thing({Trait.foo})
a = Actor()

t.accept(a)

In [ ]:
from enum import Enum

class Traits(Enum):
    foo = 1
    bar = 2
    baz = 3

s1 = set([Traits.foo, Traits.bar])

cases = {(Traits.foo,): lambda: print("case 1"),
        (Traits.bar,): lambda: print("case 2"),
        (Traits.bar, Traits.foo): lambda: print("case 3"),
        (): lambda: print("case 4"),
        (Traits.baz,): lambda: print("eh?"),
        }

for c, f in cases.items():
    if set(c) == s1:
        f()

In [ ]:
from enum import Enum
from collections import Counter
from random import choice

class Trait(Enum):
    hello = 1
    world = 2
    slut = 3
    cylon = 4
    zerg = 5
    
class ItemTrait(Enum):
    openable = 1
    closable = 2
    transparent = 3
    lockable = 4
    unlockable = 5
    lockpickable = 6
    breakable = 7

class Token(Enum):
    times = 1
    random = 2
    opened = 3
    
class Stereotype(set):
    pass

class Thing:
    __slots__ = ["__weakref__", "traits", "bag"]
    
    def __init__(self, stereotypes=[], in_traits=set(), ex_traits=set(), bag={}):
        """First set up a pile of traits (with stereotypes), then add
        all the traits that should be included. At the end,
        polish the rough edges by taking away the traits that should be 
        excluded.
        """
        self.bag = Counter(bag)
        self.traits = {t for S in stereotypes for t in S}
        self.traits.update(set(in_traits))
        self.traits.difference_update(set(ex_traits))
        
    def accept(self, actor):
        actor.handle(self)

In [ ]:
from enum import Enum
from warnings import warn
from collections import Counter



class Thing:
    #__slots__ = ["__weakref__", "traits", "bag"]
    def __init__(self, stereotypes=[], in_traits=set(), ex_traits=set()):
        """First set up a pile of traits (with stereotypes), then add
        all the traits that should be included. At the end,
        polish the rough edges by taking away the traits that should be 
        excluded.
        """
        self.traits = {t for S in stereotypes for t in S}
        self.traits.update(set(in_traits))
        self.traits.difference_update(set(ex_traits))
        
    def accept(self, actor):
        actor.handle(self)

class StateThing(Thing):
    state = State.INIT
    states = {}

    def __init__(self, states:dict, stated_traits:dict, *,
                 stereotypes=[], in_traits=set(), ex_traits=set()):
        super().__init__(stereotypes, in_traits, ex_traits)
        self.states = states
        self.stated_traits = stated_traits
    
    def flux_to(self, to_state):
        if to_state in self.states[self.state]:
            self.state = to_state
            self.traits = self.stated_traits[self.state]
        else:
            warn("Flux to %s isn't possible." % state)
    
class BaggyStateThing(StateThing):
    def __init__(self, bag:dict, states:dict, stated_traits:dict, **kwargs):
        super().__init__(states, stated_traits)
        self.bag = Counter(bag)
        
        
def thing_can(trait):
    def wrapper(method):
        def calling(self, thing):
            if trait in thing.traits:
                method(self, thing)
            else:
                warn("trait isn't available while %s is in %s" % 
                     (thing.__class__.__name__, thing.state))
        return calling
    return wrapper
    
class Guy:
    @thing_can(be.closed)
    def close(self, thing):
        thing.flux_to(State.closed)
        print("guy closes the door")
    
    @thing_can(be.entered)
    def enter(self, thing):
        print("guy walks through the door")
    
    @thing_can(be.written)
    def write(self, thing):
        print("guy writes on the door")
    
    @thing_can(be.opened)
    def open(self, thing):
        thing.flux_to(State.open)
        thing.bag[Token.opened] += 1
        print("guy opens the door")
    
    @thing_can(be.fubared)
    def fubar(self, thing):
        thing.bag[Token.hitpoint] -= 1
        print("he slams against the door")
        
        if thing.bag[Token.hitpoint] == 0:
            thing.flux_to(State.FINAL)
            print("guy broke the door")    

class Door(BaggyStateThing):
    def __init__(self):
        super().__init__(bag={Token.opened:0, Token.hitpoint:3},
                    states={State.INIT: {State.closed, State.open},
                    State.closed: {State.open, State.FINAL}, 
                    State.open: {State.closed, State.FINAL},
                    State.FINAL: {},
                    },
                    stated_traits={State.INIT: {},
                    State.open: {be.closed, be.entered, be.fubared},
                    State.closed: {be.opened, be.written, be.fubared},
                    State.FINAL: {}              
                    })
        self.flux_to(State.closed)

door = Door()

guy = Guy()
guy.open(door)
guy.fubar(door)
guy.enter(door)
guy.close(door)
guy.open(door)
guy.close(door)
guy.write(door)
guy.fubar(door)
guy.fubar(door)
guy.write(door)
door.state, door.bag

In [ ]:
from enum import Enum
from collections import Counter
from warnings import warn

class State(Enum):
    closed = 1
    open = 2
    INIT = -1
    FINAL = -2

class be(Enum):
    opened = 1
    closed = 2
    written = 3
    entered = 4
    fubared = 5

class Token(Enum):
    opened = 1
    hitpoint = 3

class StateMixin:
    def __init__(self, states:dict, stated_traits:dict, *,
                 stereotypes=[], in_traits=set(), ex_traits=set()):
        self.state = State.INIT
        self.states = {State.FINAL:{}}
        self.stated_traits = {State.INIT: set(), State.FINAL: set()}
        self.states.update(states)
        self.stated_traits.update(stated_traits)
    
    @property
    def traits(self):
        return self.stated_traits[self.state]
    
    def flux_to(self, to_state):
        if to_state in self.states[self.state]:
            self.state = to_state
        else:
            warn("Flux to %s isn't possible." % state)

class BagMixin:
    def __init__(self, bag:dict):
        self.bag = Counter(bag)

class Thing(StateMixin, BagMixin):    
    def __init__(self, bag:dict, states:dict, stated_traits:dict, **kwargs):
        StateMixin.__init__(self, states, stated_traits)
        BagMixin.__init__(self, bag)

def thing_can(trait):
    def wrapper(method):
        def calling(self, thing):
            if trait in thing.traits:
                method(self, thing)
            else:
                warn("trait isn't available while %s is in %s" % 
                     (thing.__class__.__name__, thing.state))
        return calling
    return wrapper


door = Thing(bag={Token.opened:0, Token.hitpoint:3},
             states={State.INIT: {State.closed, State.open},
                    State.closed: {State.open, State.FINAL}, 
                    State.open: {State.closed, State.FINAL},
                    },
            stated_traits={
                    State.open: {be.closed, be.entered, be.fubared},
                    State.closed: {be.opened, be.written, be.fubared},    
                    })
    
class Guy:
    @thing_can(be.closed)
    def close(self, thing):
        thing.flux_to(State.closed)
        print("guy closes the door")
    
    @thing_can(be.entered)
    def enter(self, thing):
        print("guy walks through the door")
    
    @thing_can(be.written)
    def write(self, thing):
        print("guy writes on the door")
    
    @thing_can(be.opened)
    def open(self, thing):
        thing.flux_to(State.open)
        thing.bag[Token.opened] += 1
        print("guy opens the door")
    
    @thing_can(be.fubared)
    def fubar(self, thing):
        thing.bag[Token.hitpoint] -= 1
        print("he slams against the door")
        
        if thing.bag[Token.hitpoint] == 0:
            thing.flux_to(State.FINAL)
            print("guy broke the door")  

door.flux_to(State.open)
guy = Guy()
guy.open(door)
guy.fubar(door)
guy.enter(door)
guy.close(door)
guy.open(door)
guy.close(door)
guy.write(door)
guy.fubar(door)
guy.fubar(door)
guy.write(door)
door.state, door.bag

In [ ]:
from flux.flux import Thing, can
from flux.enums.Door import IS, BE, HAS

class Guy:
    @can(BE.closed)
    def close(self, thing):
        thing.flux_to(IS.closed)
        print("guy closes the door")
    
    @can(BE.entered)
    def enter(self, thing):
        print("guy walks through the door")
    
    @can(BE.written)
    def write(self, thing):
        print("guy writes on the door")
    
    @can(BE.opened)
    def open(self, thing):
        thing.flux_to(IS.open)
        thing.bag[HAS.been_opened] += 1
        print("guy opens the door")
    
    @can(BE.fubared)
    def slam(self, thing):
        thing.bag[HAS.hitpoints] -= 1
        print("he slams against the door")
        
        if thing.bag[HAS.hitpoints] == 0:
            thing.flux_to(IS.destroyed)
            print("guy broke the door")  

door = Thing(first=IS.closed, final=IS.destroyed,
             states={IS.closed: {IS.open, IS.destroyed}, 
                    IS.open: {IS.closed, IS.destroyed},
                    },
            stateful_traits={
                    IS.open: {BE.closed, BE.entered, BE.fubared},
                    IS.closed: {BE.opened, BE.written, BE.fubared},    
                    },
            bag={HAS.been_opened:0, HAS.hitpoints:3},
            )

guy = Guy()

guy.open(door)
guy.slam(door)
guy.enter(door)
guy.close(door)
guy.open(door)
guy.close(door)
print(door.traits)
guy.write(door)
guy.slam(door)
guy.slam(door)
guy.write(door)
door.state, door.bag

In [ ]:
from flux.flux import Thing, can
from flux.enums.Door import IS, BE, HAS

class Guy:
    @can(be.closed)
    def close(self, thing):
        thing.flux_to(IS.closed)
        print("guy closes the door")
    
    @can(BE.entered)
    def enter(self, thing):
        print("guy walks through the door")
    
    @can(BE.written)
    def write(self, thing):
        print("guy writes on the door")
    
    @can(BE.opened)
    def open(self, thing):
        thing.flux_to(IS.open)
        thing.bag[HAS.opened] += 1
        print("guy opens the door")
    
    @can(Door.be.fubared)
    def slam(self, thing):
        thing.bag[Door.Token.hitpoint] -= 1
        print("he slams against the door")
        
        if thing.bag[Door.Token.hitpoint] == 0:
            thing.flux_to(Door.iss.gone)
            print("guy broke the door")  

door = Thing(first=Door.iss.closed, final=Door.iss.gone,
             states={Door.iss.closed: {Door.iss.open, Door.iss.gone}, 
                    Door.iss.open: {Door.iss.closed, Door.iss.gone},
                    },
            stateful_traits={
                    Door.iss.open: {Door.be.closed, Door.be.entered, Door.be.fubared},
                    Door.iss.closed: {Door.be.opened, Door.be.written, Door.be.fubared},
                    Door.iss.gone: {Door.be.entered}
                    },
            bag={Door.Token.opened:0, Door.Token.hitpoint:3},
            )

guy = Guy()

guy.open(door)
guy.slam(door)
guy.enter(door)
guy.close(door)
guy.open(door)
guy.close(door)
guy.write(door)
guy.slam(door)
guy.slam(door)
guy.write(door)
door.state, door.bag

In [ ]:
from flux.flux import Thing
from flux.enums import Markov, Token
from random import random

System = Thing(first=Markov.two, final=Markov.down, 
               states={Markov.two:{Markov.one,},
                       Markov.one: {Markov.two, Markov.down}
                                   },
              bag={Token.flux:0}
              )

while System.state is not Markov.down:
    print(1)
    if random() < 0.3:
        System.flux_to(Markov.one) or System.flux_to(Markov.two)
        System.bag[Token.flux] += 1
    elif System.state is Markov.one and random() < 0.1:
        System.flux_to(Markov.down)
 

System.bag[Token.flux]

Since we figured out how to dispatch on different sets of traits, let's see how easy it is to implement the stateful GUI of PL.
Okay, it seems there's an even more elegant way than using dispatch here. If the states are implemented as classes - not enums - it's possible to simply use @staticmethod for the states and have all problems fixed at the same time: 1) The state definition is a well-named singleton with auto-completion 2) it's trivial to define enter() and exit() for each state and call them on every flux (simply override flux_to) 3) it's easy to "dispatch" methods per state - states don't need to be instanciated but can be decorated with @staticmethod so they can be called just like that - exactly the way the dispatch would work.

This is an even more elegant solution since it takes care of dispatching state-dependent methods with no boilerplate at all. The state transition definition is clean and easy as it should be while it's still trivial to have state-dependent traits as flags and the state manager also has a bag to keep track of any additional info.

In [ ]:
import logging

base = None
render = None
logger = logging.getLogger('STATE')


class StateManager(object):
    def __init__(self, bridge):
        global base, render
        base = bridge
        render = bridge.render
        self.state = None
    
        self.available_states = {
            "loading": Loading(),
            "login": Login(),
            "signup": Signup(),
            "playing": Playing(),
            "exit": Exit(),
            }

    def request_switch(self, state_name):
        """Try to switch to the requested state."""
        logger.info(state_name)
        # The very beginning
        if self.state is None:
            self.state = self.available_states[state_name]
            self.state.enter()
        elif state_name in self.state.valid_transitions:
            self.state.exit()
            self.state = self.available_states[state_name]
            self.state.enter()
        else:
            raise RuntimeWarning
            
    @property
    def name(self):
        return self.__class__.__name__
            
    def display_message(self, message):
        self.state.display_message(message)
            
            
class State(object):
    """I represent the inner state of the GUI.
    
    Implementing the state pattern, the behaviour of the GUI
    depends on its state (what is focused, playing, logged in, etc.).
    The state object provides a unified interface, the subclasses implement
    different behaviours. The GUI sets its state accordingly.
    """
    # States that may be switched to from here
    valid_transitions = []
    
    def display_message(self, message):
        """Template for displaying messages in different situations."""
        raise NotImplementedError
        
    def enter(self):
        """Set special stuff when switching to this state.
        Always called on a state switch. May or may not be implemented.
        """
        pass
    
    def exit(self):
        """Reset special stuff when leaving this state.
        
        Always called on a state switch. May or may not be implemented.
        """
        pass
    
    
class Loading(State):
    """The GUI conditions while starting the game."""
    valid_transitions = ["login"]
    
    def enter(self):
        render.hide()
        base.setBackgroundColor(1, 1, 1)
        base.setFrameRateMeter(False)

    def display_message(self, message):
        logger.info(message.m_body)


class Login(State):
    """Logging in."""
    valid_transitions = ["signup", "playing", "exit"]
     
    def enter(self):
        base.gui.show()
        base.gui.System.setGUISheet(base.gui.login.root)
     
    def exit(self):
        base.gui.System.setGUISheet(None)
     
    def display_message(self, message):
        logger.info(message.m_body)
        
        # switch/case and call of the function with the message
        try:
            {"motd": base.gui.login.set_motd,
            "error": base.gui.login.set_error,
            "news": base.gui.login.set_message,
            "name": base.gui.login.set_name,
            "status": base.gui.login.set_status,
            }[message.m_type](message)
        except KeyError:
            logger.error("Message can't be handled':" + message.m_body)


class Signup(State):
    """Making a new account."""
    valid_transitions = ["login"]
    playing = False
    navigating = False

    def enter(self):
        # UGLY
        base.gui.System.setGUISheet(base.gui.signup.root)

    def exit(self):
        base.gui.System.setGUISheet(None)
    
    def display_message(self, message):
        """Display warnings and other things without annoying."""
        
        try:
            {"ok": lambda m: base.gui.signup.set_message(m.m_body),
            }[message.m_type](message)
        except KeyError:
            logger.error("Message can't be handled':" + message.m_body)


class Playing(State):
    """Normal gameplay."""
    valid_transitions = ["exit"]
    playing = True
    navigating = True
    
    def enter(self):
        base.setBackgroundColor(0.5, 0.5, 0.5)
        render.show()
        base.setFrameRateMeter(True)

        base.gui.System.setGUISheet(base.gui.ingame.root)

    def exit(self):
        base.gui.System.setGUISheet(None)
        
    def display_message(self, message):
        ## hack = message.m_body.decode("CP1252")
        
        ## if message.m_type == "system":
            ## line = "\nSystem: {0.m_body}".format(message)
    
        ## if message.m_type == "chat":
            ## if hack.startswith("/me"):
                ## hack = message.m_body[4:]
                ## line = "\n{0.m_from} {1}".format(message, hack)
            ## else:
                ## line = "\n{0.m_from}@{0.m_to}: {1}".format(message, hack)
    
        base.gui.ingame.display_message(message)


class Exit(State):
    """Exiting and clean up."""
    def enter(self):
        base.careen()


In [ ]:
from flux.flux import TraitMachine
from flux.enums import GUI
from random import random

class ContextBag(TraitMachine):
    def flux_to(self, state):
        self.state.exit()
        super().flux_to(state)
        self.state.enter()

class State:
    @staticmethod
    def enter():
        pass
    
    @staticmethod
    def exit():
        pass
    
    @staticmethod
    def display_message(msg):
        print("default display", msg)

class LOADING(State):
    @staticmethod
    def enter():
        print("loading entered..")
#         render.hide()
#         base.setBackgroundColor(1, 1, 1)
#         base.setFrameRateMeter(False)

    @staticmethod
    def display_message(message):
        print(message)
        logger.info(message.m_body)

class LOGIN(State):
    @staticmethod
    def enter():
        print("login entered")
        #base.gui.System.setGUISheet(base.gui.signup.root)

    @staticmethod
    def exit():
        print("login exited")
#         base.gui.System.setGUISheet(None)

    @staticmethod
    def display_message(message):
        print("displaying message on login:", message)
        #        try:
#             {"ok": lambda m: base.gui.signup.set_message(m.m_body),
#             }[message.m_type](message)
#         except KeyError:
#             logger.error("Message can't be handled':" + message.m_body)

class EXITING(State):
    pass

class SIGNUP(State):
    pass

class PLAYING(State):
    pass

win = ContextBag(first=LOADING, final=EXITING,
           states={LOADING: {LOGIN},
                  LOGIN: {SIGNUP, PLAYING, EXITING},
                  SIGNUP: {PLAYING, EXITING},
                  PLAYING: {EXITING}},
            in_traits=[GUI.BE.closed],
           stateful_traits={PLAYING: {GUI.CAN.navigate, GUI.CAN.play}},
           bag={GUI.HAS.clicks:0})
            
win.flux_to(LOGIN)
win.state.display_message("omg!")
win.flux_to(PLAYING)
win.traits

In [ ]:
from flux.flux import Thing
from flux.enums import PL

item = Thing(first=PL.IS.working, final=PL.IS.destroyed,
            states={PL.IS.working:{PL.IS.destroyed}},
            stateful_traits={PL.IS.working:{PL.BE.destroyed}})

item2 = item.clone()

item2

In [ ]:
from flux.flux import TraitBag, can
from enum import Enum

IS = Enum("is", "closed open destroyed pressed released")
BE = Enum("be", "opened closed entered written fubared")
HAS = Enum("has", "been_opened hitpoints")

class Guy:
    @can(BE.closed)
    def close(self, thing):
        thing.flux_to(IS.closed)
        print("guy closes the door")
    
    @can(BE.entered)
    def enter(self, thing):
        print("guy walks through the door")
    
    @can(BE.written)
    def write(self, thing):
        print("guy writes on the door")
    
    @can(BE.opened)
    def open(self, thing):
        thing.flux_to(IS.open)
        thing.bag[HAS.been_opened] += 1
        print("guy opens the door")
    
    @can(BE.fubared)
    def slam(self, thing):
        thing.bag[HAS.hitpoints] -= 1
        print("he slams against the %s" % thing)
        
        if thing.bag[HAS.hitpoints] == 0:
            thing.flux_to(IS.destroyed)
            print("guy broke the door")  


handle = Handle(name="handle", first=IS.released, final=IS.destroyed,
               states={IS.released: {IS.pressed, IS.destroyed},
                      IS.pressed: {IS.released, IS.destroyed}
                      },
               stateful_traits={IS.released:{BE.opened},
                               IS.pressed:{BE.closed}
                               }
                )

door = Thing(name="door", first=IS.closed, final=IS.destroyed,
             states={IS.closed: {IS.open, IS.destroyed}, 
                    IS.open: {IS.closed, IS.destroyed},
                    },
            stateful_traits={
                    IS.open: {BE.closed, BE.entered, BE.fubared},
                    IS.closed: {BE.opened, BE.written, BE.fubared},    
                    },
            bag={HAS.been_opened:0, HAS.hitpoints:3},
            children=[handle],
            )

   

guy = Guy()


guy.open(door)
guy.slam(door)
guy.enter(door)
guy.close(door)
guy.open(door)
guy.close(door)
print(door.traits)
guy.write(door)
guy.slam(door)
guy.slam(door)
guy.write(door)
door.state, door.bag

In [ ]:
from enum import Enum
from flux.flux import can

IS = Enum("IS", "working, destroyed, done, ready, broken") 
BE = Enum("BE", "selected, destroyed, used, repaired, cancelled")
HAS = Enum("HAS", "amount")
OF = Enum("OF", "water, meat, paprica")

@can(BE.destroyed)
def destroy(self, thing):
    thing.flux_to(IS.destroyed)
    print("%s broke the %s!!!" % (self.__class__.__name__, thing.name))

In [ ]:
from enum import Enum

class IS(Enum):
	starting = 1
	connecting = 2
	ready = 3
	disconnected = 4
	down = 5

class BE(Enum):
	prepared = 1
	etc = 2

In [ ]:
from enum import Enum

IS = Enum("STATE", "loading login signup playing exiting")
CAN = Enum("CAN", "start navigate play configure")
BE  = Enum("BE", "closed clicked")
HAS = Enum("HAS", "clicks")

In [ ]:
from enum import Enum

IS = Enum('IS', 'closed, open, destroyed')
BE = Enum("BE", "opened, closed, written, entered, fubared")
HAS = Enum("HAS", "been_opened, hitpoints, been_fluxed")